In [1]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    del data['context_timestamp_tmp']


#     print('predict_category_property_ing_0')
#     for i in range(3):
#         data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
#         )
    for i in range(3):
        temp = data[['user_occupation_id','property_%d'%(i),'instance_id']].groupby(['user_occupation_id','property_%d'%(i)]).aggregate('count')
        temp1 = temp.reset_index()
        temp2 = temp.groupby('user_occupation_id').sum().reset_index()
        temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
        temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
        temp3['property_%dpv'%(i)] = temp4
        temp3.drop('instance_id_x', axis=1, inplace=True)
        temp3.drop('instance_id_y', axis=1, inplace=True)
        data = pd.merge(data,temp3,on=['user_occupation_id','property_%d'%(i)],how='left')
    for i in range(1,3):
        temp = data[['user_occupation_id','category_%d'%(i),'instance_id']].groupby(['user_occupation_id','category_%d'%(i)]).aggregate('count')
        temp1 = temp.reset_index()
        temp2 = temp.groupby('user_occupation_id').sum().reset_index()
        temp3 = pd.merge(temp1,temp2,on=['user_occupation_id'],how='left')
        temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
        temp3['category_%dpv'%(i)] = temp4
        temp3.drop('instance_id_x', axis=1, inplace=True)
        temp3.drop('instance_id_y', axis=1, inplace=True)
        data = pd.merge(data,temp3,on=['user_occupation_id','category_%d'%(i)],how='left')
    for i in range(3):
        temp = data[['user_gender_id','property_%d'%(i),'instance_id']].groupby(['user_gender_id','property_%d'%(i)]).aggregate('count')
        temp1 = temp.reset_index()
        temp2 = temp.groupby('user_gender_id').sum().reset_index()
        temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
        temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
        temp3['property_%dpv'%(i)] = temp4
        temp3.drop('instance_id_x', axis=1, inplace=True)
        temp3.drop('instance_id_y', axis=1, inplace=True)
        data = pd.merge(data,temp3,on=['user_gender_id','property_%d'%(i)],how='left')
    for i in range(1,3):
        temp = data[['user_gender_id','category_%d'%(i),'instance_id']].groupby(['user_gender_id','category_%d'%(i)]).aggregate('count')
        temp1 = temp.reset_index()
        temp2 = temp.groupby('user_gender_id').sum().reset_index()
        temp3 = pd.merge(temp1,temp2,on=['user_gender_id'],how='left')
        temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
        temp3['category_%dpv'%(i)] = temp4
        temp3.drop('instance_id_x', axis=1, inplace=True)
        temp3.drop('instance_id_y', axis=1, inplace=True)
        data = pd.merge(data,temp3,on=['user_gender_id','category_%d'%(i)],how='left')
    temp = data[['user_age_level','item_price_level','instance_id']].groupby(['user_age_level','item_price_level']).aggregate('count')
    temp1 = temp.reset_index()
    temp2 = temp.groupby('user_age_level').sum().reset_index()
    temp3 = pd.merge(temp1,temp2,on=['user_age_level'],how='left')
    temp4 = temp3[['instance_id_x']].rename(columns=({'instance_id_x':'sum'}))/temp3[['instance_id_y']].rename(columns=({'instance_id_y':'sum'}))
    temp3['age_pv'] = temp4
    temp3.drop('instance_id_x', axis=1, inplace=True)
    temp3.drop('instance_id_y', axis=1, inplace=True)
    data = pd.merge(data,temp3,on=['user_age_level','user_age_level'],how='left')
    del(temp3)
    del(temp2)
    del(temp1)
    del(temp4)
    del(temp)
#     print('predict_category_property_ing_1')
#     for i in range(3):
#         data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
#             lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
#         )
    
#         for j in range(3):
#             data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
#                 lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
#             )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data

In [2]:
print('train')
train = pd.read_csv('train.txt',sep=" ")
data2 = pd.read_csv('1234.csv',index_col=False) 
datac = pd.read_csv('clickpv.csv',index_col=False) 
train.insert(1, 'item_category_list_num', data2[['item_category_list']])
train.insert(2, 'item_property_list_num', data2[['item_property_list']])
train.insert(1, 'clickpv', datac[[ 'clickpv']])
train = pre_process(train)

all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
all_shape (5749977, 47)
2018-09-24 23:59:47
(2495745, 47)
(1456376, 47)


In [3]:
train.columns

Index(['instance_id', 'clickpv', 'item_category_list_num',
       'item_property_list_num', 'item_id', 'item_brand_id', 'item_city_id',
       'item_price_level_x', 'item_sales_level', 'item_collected_level',
       'item_pv_level', 'user_id', 'user_gender_id', 'user_age_level',
       'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'is_trade', 'category_0', 'category_1', 'category_2', 'property_0',
       'property_1', 'property_2', 'week', 'hour', 'property_0pv_x',
       'property_1pv_x', 'property_2pv_x', 'category_1pv_x', 'category_2pv_x',
       'property_0pv_y', 'property_1pv_y', 'property_2pv_y', 'category_1pv_y',
       'category_2pv_y', 'item_price_level_y', 'age_pv'],
      dtype='object')

In [4]:
print('test')
test_a = pd.read_csv('test.txt',sep=" ")
data3 = pd.read_csv('456.csv',index_col=False)
data2c = pd.read_csv('test_clickpv.csv',index_col=False) 
# data2.pop('item_category_list')
test_a.insert(1, 'item_category_list_num', data3[['item_category_list']])
# data2.pop('item_property_list')
test_a.insert(2, 'item_property_list_num', data3[['item_property_list']])
test_a.insert(1, 'clickpv', data2c[[ 'clickpv']])
print(test_a.shape)
test_a = pre_process(test_a)

test
(18371, 29)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time


In [5]:
test_a

,instance_id,clickpv,item_category_list_num,item_property_list_num,item_id,item_brand_id,item_city_id,item_price_level_x,item_sales_level,item_collected_level,...,property_2pv_x,category_1pv_x,category_2pv_x,property_0pv_y,property_1pv_y,property_2pv_y,category_1pv_y,category_2pv_y,item_price_level_y,age_pv
0,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,1,0.000376
1,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,2,0.001127
2,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,3,0.007135
3,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,4,0.040556
4,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,5,0.110026
5,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,6,0.251220
6,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,7,0.408562
7,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,8,0.176868
8,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,0.027666,0.344102,0.996821,0.142567,0.007041,0.026060,0.333333,0.995572,9,0.004131
9,398316874173557226,0.01,2,0,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,0.098490,0.234184,0.993170,0.505102,0.445489,0.112245,0.249194,0.995166,2,0.000399


In [6]:
# 这里是增加的内容
import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count


In [7]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)

train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)
y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')



2018-09-20 00:00:01
2018-09-18 00:00:01
1 days 23:59:54
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43


In [8]:
print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data



(161570, 47)


In [9]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
feat_set = list(test_a.columns)
for i,feat in enumerate(feat_set):
    tmp = lb.fit_transform((list(train[feat])+list(val[feat])+list(test_a[feat])))
    print(tmp)
    enc.fit(tmp.reshape(-1,1))
    x_train = enc.transform(lb.transform(train[feat]).reshape(-1, 1))
    x_test = enc.transform(lb.transform(test_a[feat]).reshape(-1, 1))
    x_val = enc.transform(lb.transform(val[feat]).reshape(-1, 1))
    if i == 0:
        X_train, X_test,X_val = x_train, x_test,x_val
    else:
        X_train, X_test,X_val = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test)),sparse.hstack((X_val, x_val))




baseline ing ... ...
Index(['clickpv', 'item_category_list_num', 'item_property_list_num',
       'item_id', 'item_brand_id', 'item_city_id', 'item_price_level_x',
       'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id',
       'user_gender_id', 'user_age_level', 'user_occupation_id',
       'user_star_level', 'context_id', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'category_0', 'category_1', 'category_2', 'property_0', 'property_1',
       'property_2', 'week', 'hour', 'property_0pv_x', 'property_1pv_x',
       'property_2pv_x', 'category_1pv_x', 'category_2pv_x', 'property_0pv_y',
       'property_1pv_y', 'property_2pv_y', 'category_1pv_y', 'category_2pv_y',
       'item_price_level_y', 'age_pv', 'user_count', 'item_count'],
      dtype='object')
[0 0 0 ..., 0 0 0]
[2 2 2 ..., 2 2 2]
[2 2 2 ..., 1 1 1]
[3411 341

In [18]:
lr = LogisticRegression(C = 1,penalty='l1')


lr.fit(X_train, y_train)
proba_val = lr.predict_proba(X_val)[:,1]
proba_sub = lr.predict_proba(X_test)[:,1]

In [19]:
print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print(log_loss(y_val,proba_val))

0.00325040696511
0.104454423314


In [ ]:
X_train.shape

(2495745, 569206)

In [ ]:

import lightgbm as lgb

gbm = lgb.LGBMClassifier(objective='binary',
                        num_leaves=64,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.75,
                        reg_alpha = 0.4
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)

[1]	valid_0's binary_logloss: 0.683883
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.6748
[3]	valid_0's binary_logloss: 0.665894
[4]	valid_0's binary_logloss: 0.657164
[5]	valid_0's binary_logloss: 0.6486
[6]	valid_0's binary_logloss: 0.640195
[7]	valid_0's binary_logloss: 0.631951
[8]	valid_0's binary_logloss: 0.623864
[9]	valid_0's binary_logloss: 0.615926
[10]	valid_0's binary_logloss: 0.608135
[11]	valid_0's binary_logloss: 0.600489
[12]	valid_0's binary_logloss: 0.592979
[13]	valid_0's binary_logloss: 0.585607
[14]	valid_0's binary_logloss: 0.578365
[15]	valid_0's binary_logloss: 0.571255
[16]	valid_0's binary_logloss: 0.564268
[17]	valid_0's binary_logloss: 0.557403
[18]	valid_0's binary_logloss: 0.55066
[19]	valid_0's binary_logloss: 0.544033
[20]	valid_0's binary_logloss: 0.537522
[21]	valid_0's binary_logloss: 0.531121
[22]	valid_0's binary_logloss: 0.524828
[23]	valid_0's binary_logloss: 0.518641
[24]	valid_0's binary_logloss: 

[203]	valid_0's binary_logloss: 0.125045
[204]	valid_0's binary_logloss: 0.124499
[205]	valid_0's binary_logloss: 0.123962
[206]	valid_0's binary_logloss: 0.123431
[207]	valid_0's binary_logloss: 0.122906
[208]	valid_0's binary_logloss: 0.122387
[209]	valid_0's binary_logloss: 0.121875
[210]	valid_0's binary_logloss: 0.121369
[211]	valid_0's binary_logloss: 0.12087
[212]	valid_0's binary_logloss: 0.120376
[213]	valid_0's binary_logloss: 0.119889
[214]	valid_0's binary_logloss: 0.119407
[215]	valid_0's binary_logloss: 0.118933
[216]	valid_0's binary_logloss: 0.118464
[217]	valid_0's binary_logloss: 0.118001
[218]	valid_0's binary_logloss: 0.117546
[219]	valid_0's binary_logloss: 0.117093
[220]	valid_0's binary_logloss: 0.116648
[221]	valid_0's binary_logloss: 0.116209
[222]	valid_0's binary_logloss: 0.115774
[223]	valid_0's binary_logloss: 0.115346
[224]	valid_0's binary_logloss: 0.114924
[225]	valid_0's binary_logloss: 0.114504
[226]	valid_0's binary_logloss: 0.114091
[227]	valid_0's b

[401]	valid_0's binary_logloss: 0.0863066
[402]	valid_0's binary_logloss: 0.0862704
[403]	valid_0's binary_logloss: 0.086234
[404]	valid_0's binary_logloss: 0.0861987
[405]	valid_0's binary_logloss: 0.0861621
[406]	valid_0's binary_logloss: 0.086129
[407]	valid_0's binary_logloss: 0.0860958
[408]	valid_0's binary_logloss: 0.0860615
[409]	valid_0's binary_logloss: 0.086028
[410]	valid_0's binary_logloss: 0.0859962
[411]	valid_0's binary_logloss: 0.0859645
[412]	valid_0's binary_logloss: 0.0859313
[413]	valid_0's binary_logloss: 0.0858999
[414]	valid_0's binary_logloss: 0.0858696
[415]	valid_0's binary_logloss: 0.0858402
[416]	valid_0's binary_logloss: 0.0858115
[417]	valid_0's binary_logloss: 0.0857822
[418]	valid_0's binary_logloss: 0.0857528
[419]	valid_0's binary_logloss: 0.0857251
[420]	valid_0's binary_logloss: 0.085698
[421]	valid_0's binary_logloss: 0.0856698
[422]	valid_0's binary_logloss: 0.0856408
[423]	valid_0's binary_logloss: 0.0856154
[424]	valid_0's binary_logloss: 0.0855

[597]	valid_0's binary_logloss: 0.0837538
[598]	valid_0's binary_logloss: 0.083752
[599]	valid_0's binary_logloss: 0.0837485
[600]	valid_0's binary_logloss: 0.0837454
[601]	valid_0's binary_logloss: 0.0837439
[602]	valid_0's binary_logloss: 0.0837427
[603]	valid_0's binary_logloss: 0.0837392
[604]	valid_0's binary_logloss: 0.0837352
[605]	valid_0's binary_logloss: 0.0837321
[606]	valid_0's binary_logloss: 0.0837301
[607]	valid_0's binary_logloss: 0.0837256
[608]	valid_0's binary_logloss: 0.0837233
[609]	valid_0's binary_logloss: 0.0837178
[610]	valid_0's binary_logloss: 0.0837149
[611]	valid_0's binary_logloss: 0.0837116
[612]	valid_0's binary_logloss: 0.0837086
[613]	valid_0's binary_logloss: 0.083701
[614]	valid_0's binary_logloss: 0.0836983
[615]	valid_0's binary_logloss: 0.0836944
[616]	valid_0's binary_logloss: 0.0836909
[617]	valid_0's binary_logloss: 0.083688
[618]	valid_0's binary_logloss: 0.0836841
[619]	valid_0's binary_logloss: 0.0836822
[620]	valid_0's binary_logloss: 0.083

[793]	valid_0's binary_logloss: 0.083406
[794]	valid_0's binary_logloss: 0.0834027
[795]	valid_0's binary_logloss: 0.0834012
[796]	valid_0's binary_logloss: 0.0833989
[797]	valid_0's binary_logloss: 0.0833986
[798]	valid_0's binary_logloss: 0.0833975
[799]	valid_0's binary_logloss: 0.0833955
[800]	valid_0's binary_logloss: 0.0833955
[801]	valid_0's binary_logloss: 0.0833956
[802]	valid_0's binary_logloss: 0.083395
[803]	valid_0's binary_logloss: 0.083393
[804]	valid_0's binary_logloss: 0.0833925
[805]	valid_0's binary_logloss: 0.0833928
[806]	valid_0's binary_logloss: 0.0833915
[807]	valid_0's binary_logloss: 0.0833896
[808]	valid_0's binary_logloss: 0.0833883
[809]	valid_0's binary_logloss: 0.0833865
[810]	valid_0's binary_logloss: 0.0833859
[811]	valid_0's binary_logloss: 0.0833853
[812]	valid_0's binary_logloss: 0.083384
[813]	valid_0's binary_logloss: 0.0833841
[814]	valid_0's binary_logloss: 0.0833833
[815]	valid_0's binary_logloss: 0.0833825
[816]	valid_0's binary_logloss: 0.0833

[989]	valid_0's binary_logloss: 0.0832749
[990]	valid_0's binary_logloss: 0.0832747
[991]	valid_0's binary_logloss: 0.0832741
[992]	valid_0's binary_logloss: 0.083275
[993]	valid_0's binary_logloss: 0.0832747
[994]	valid_0's binary_logloss: 0.0832742
[995]	valid_0's binary_logloss: 0.0832735
[996]	valid_0's binary_logloss: 0.0832718
[997]	valid_0's binary_logloss: 0.0832728
[998]	valid_0's binary_logloss: 0.083272
[999]	valid_0's binary_logloss: 0.0832712
[1000]	valid_0's binary_logloss: 0.0832692
[1001]	valid_0's binary_logloss: 0.0832694
[1002]	valid_0's binary_logloss: 0.0832698
[1003]	valid_0's binary_logloss: 0.08327
[1004]	valid_0's binary_logloss: 0.0832693
[1005]	valid_0's binary_logloss: 0.0832683
[1006]	valid_0's binary_logloss: 0.0832681
[1007]	valid_0's binary_logloss: 0.0832655
[1008]	valid_0's binary_logloss: 0.083266
[1009]	valid_0's binary_logloss: 0.0832651
[1010]	valid_0's binary_logloss: 0.0832657
[1011]	valid_0's binary_logloss: 0.0832663
[1012]	valid_0's binary_log

[1181]	valid_0's binary_logloss: 0.0832133
[1182]	valid_0's binary_logloss: 0.0832133
[1183]	valid_0's binary_logloss: 0.0832138
[1184]	valid_0's binary_logloss: 0.0832142
[1185]	valid_0's binary_logloss: 0.0832145
[1186]	valid_0's binary_logloss: 0.0832142
[1187]	valid_0's binary_logloss: 0.0832146
[1188]	valid_0's binary_logloss: 0.0832144
[1189]	valid_0's binary_logloss: 0.0832149
[1190]	valid_0's binary_logloss: 0.0832166
[1191]	valid_0's binary_logloss: 0.0832155
[1192]	valid_0's binary_logloss: 0.0832148
[1193]	valid_0's binary_logloss: 0.0832142
[1194]	valid_0's binary_logloss: 0.083214
[1195]	valid_0's binary_logloss: 0.0832144
[1196]	valid_0's binary_logloss: 0.0832149
[1197]	valid_0's binary_logloss: 0.0832147
[1198]	valid_0's binary_logloss: 0.0832136
[1199]	valid_0's binary_logloss: 0.0832137
[1200]	valid_0's binary_logloss: 0.0832134
[1201]	valid_0's binary_logloss: 0.0832132
[1202]	valid_0's binary_logloss: 0.0832133
[1203]	valid_0's binary_logloss: 0.0832135
[1204]	valid

In [ ]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict_proba(X_val, num_iteration=gbm.best_iteration_ )[:,1]
y_sub_1 = gbm.predict_proba(X_test, num_iteration=gbm.best_iteration_ )[:,1]
print()
print(log_loss(y_train,gbm.predict_proba(X_train)[:,1]))
print(log_loss(y_val,y_pred_1))

print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.0831080714914

In [299]:
# import xgboost as xgb

# model_xgb = xgb.XGBClassifier( n_estimators=2000,random_state =666, nthread = -1,n_jobs=-1)
# model_xgb.fit(X_train, y_train,eval_metric='logloss')


# y_xpred_1 = model_xgb.predict_proba(X_val)[:,1]
# y_xsub_1 = model_xgb.predict_proba(X_test)[:,1]


In [291]:
# print(log_loss(y_train,model_xgb.predict_proba(X_train)))
# print(log_loss(y_val, y_xpred_1))

In [292]:
import numpy as np
bes = 1
for i in np.arange(0,1,0.01):
    if(log_loss(y_val,(i * y_pred_1 + (1-i) * proba_val)) < bes):
        bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
print(bes)#0.083131236937

0.0847360440304


In [293]:
xx_analyse = pd.DataFrame()
xx_analyse['ture'] = list(y_val)
xx_analyse['pre'] = list(proba_val)
xx_analyse['pre_1'] = list(y_pred_1)
xx_analyse.to_csv('temp.csv',index=False)


sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180322.txt',sep=" ",index=False)

In [294]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list((y_sub_1 + proba_sub)/2)
sub.to_csv('20180324.txt',sep=" ",index=False)

In [ ]:
(proba_sub + y_sub_1)/2